In [1]:
from transformer import Transformer
import silence_tensorflow.auto
import numpy as np

N_TIMESTEPS = 5
MAX_N_HARPS = 5
N_FEATURES = 34
INPUT_DIM = (N_TIMESTEPS, MAX_N_HARPS, N_FEATURES)
N_OUT = 2
OUTPUT_DIM =(N_TIMESTEPS,N_OUT)
NUM_LAYERS = 3
D_MODEL = 12
DFF = 24
NUM_HEADS = 3
RATE = 0.1

assert N_FEATURES > D_MODEL
assert D_MODEL % NUM_HEADS == 0

t = Transformer(
    num_layers=NUM_LAYERS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dff=DFF,
    input_dimensions=INPUT_DIM,
    target_dimensions=OUTPUT_DIM,
    rate=RATE,
)


In [40]:
out_data = pd.read_csv("/mnt/hackathon2021/Weltraumwetterlage/train_data/output.csv")
out_data["Long"] = np.maximum(out_data["Gp_Long"].to_numpy(), out_data["Gs_Long"].to_numpy())
out_data["Short"] = np.maximum(out_data["Gp_Short"].to_numpy(), out_data["Gs_Short"].to_numpy())
out_data = out_data[["timestamp", "Long", "Short"]]
xray_df = out_data
xray_df = xray_df[(xray_df["Short"] > 0.0) & (xray_df["Long"] > 0.0)]

xray_columns = xray_df.columns
metadata["xray_mean"] = xray_df[xray_columns[1:]].mean().to_numpy()
metadata["xray_std"] = xray_df[xray_columns[1:]].std().to_numpy()

# balance input data 
much_boom_bang = xray_df[xray_df["Long"] >= 1.0e-7]
less_bomm_bang = xray_df[(xray_df["Long"] < 1.0e-7)].head(len(much_boom_bang))
ts_much_boom_bang = much_boom_bang["timestamp"]
ts_less_bomm_bang = less_bomm_bang["timestamp"]
x_ray_timestamps = ts_much_boom_bang.append(ts_less_bomm_bang)
xray_df = much_boom_bang.append(less_bomm_bang)

xray_df_notime = xray_df[xray_columns[1:]]
xray_df_notime -= metadata["xray_mean"]
xray_df_notime += metadata["xray_std"]
xray_df_notime["timestamp"] = x_ray_timestamps
xray_df = xray_df_notime
xray_df = xray_df.sort_values("timestamp")
x_ray_timestamps = x_ray_timestamps.sort_values()
x_ray_timestamps



sharp_df = pd.read_csv("/mnt/hackathon2021/Weltraumwetterlage/train_data/input.csv")
sharp_columns = sharp_df.columns
metadata["sharp_mean"] = sharp_df[sharp_columns[1:]].mean().to_numpy()
metadata["sharp_std"] = sharp_df[sharp_columns[1:]].std().to_numpy()

# balance input data
sharp_df["keep"] = sharp_df["timestamp"].apply(lambda x: x in x_ray_timestamps.values)
sharp_df = sharp_df[sharp_df["keep"] == False]
sharp_df_notime = sharp_df.drop(columns=["keep", "timestamp"])

sharp_df_notime -= metadata["sharp_mean"]
sharp_df_notime /= metadata["sharp_std"]
sharp_df_notime["timestamp"] = sharp_df["timestamp"]
sharp_df = sharp_df_notime

,harp,USFLUX,MEANGAM,MEANGBT,MEANGBZ,MEANGBH,TOTPOT,TOTUSJZ,TOTUSJH,ABSNJZH,SAVNCPP,MEANPOT,MEANSHR,SHRGT45,SIZE,SIZE_ACR,NACR,NPIX,MEANJZD,MEANALP,MEANJZH,Gp_Long,Gp_Short,Gs_Long,Gs_Short,Electron 38-53,Electron 175-315,Protons keV S,Protons keV 47-68,Protons keV 115-195,Protons keV 310-580,Protons keV 795-1193,Protons keV 1060-1900
22780,6983,1.288015e+22,30.656,74.054,76.720,33.580,6.133272e+22,1.091607e+13,561.064,72.324,4.011518e+12,2371.507,23.544,10.121,1761.885376,163.413834,1851,19957,0.000000,0.010926,0.003714,7.630000e-08,1.060000e-08,7.200000e-08,1.160000e-08,873.0,25.2,0.0,2160.0,17.2,2.28,0.765,0.140
22781,6982,2.613419e+21,29.284,126.471,126.000,45.050,1.021131e+22,3.765157e+12,163.447,6.233,1.456004e+12,1733.315,23.772,5.523,3727.707031,265.117096,3003,42224,0.621386,0.000000,0.000000,7.630000e-08,1.060000e-08,7.200000e-08,1.160000e-08,873.0,25.2,0.0,2160.0,17.2,2.28,0.765,0.140
22782,6983,1.294440e+22,30.356,74.386,77.264,33.843,6.197850e+22,1.114402e+13,575.914,71.344,3.480686e+12,2390.095,23.398,9.244,1771.416382,173.124725,1961,20065,0.000000,0.010733,0.003654,7.540000e-08,1.160000e-08,6.810000e-08,1.390000e-08,1130.0,26.1,0.0,2140.0,17.2,2.52,0.715,0.125
22783,6982,2.560015e+21,29.395,127.472,126.782,45.614,1.009053e+22,3.737725e+12,161.361,6.292,1.599460e+12,1764.113,23.854,6.617,3715.074707,263.880585,2989,42081,0.703059,0.000000,0.000000,7.540000e-08,1.160000e-08,6.810000e-08,1.390000e-08,1130.0,26.1,0.0,2140.0,17.2,2.52,0.715,0.125
22784,6983,1.290127e+22,30.491,74.086,76.911,33.944,6.193475e+22,1.107530e+13,568.607,75.717,4.024586e+12,2393.557,23.476,9.372,1779.084351,169.327301,1918,20152,0.000000,0.011424,0.003887,7.530000e-08,1.490000e-08,7.440000e-08,1.240000e-08,1100.0,29.5,0.0,2020.0,18.7,2.23,0.770,0.163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112874,7399,3.834525e+20,25.727,128.853,129.800,35.670,1.453782e+21,5.205354e+11,22.354,0.094,1.903186e+11,1718.492,24.130,5.024,356.484924,25.821161,304,4197,0.000000,0.000000,0.000000,7.900000e-08,3.380000e-08,3.890000e-08,2.580000e-08,1210.0,36.2,0.0,4190.0,24.4,2.88,0.598,0.144
112875,7399,3.632628e+20,25.546,128.163,130.590,35.234,1.361426e+21,5.083623e+11,22.328,0.637,2.366173e+11,1746.399,23.287,3.748,352.235474,25.226414,297,4147,0.116347,0.004360,0.001086,8.050000e-08,3.420000e-08,3.990000e-08,2.640000e-08,1240.0,29.8,0.0,4300.0,25.2,2.38,0.684,0.259
112876,7399,3.595034e+20,25.603,126.500,128.659,36.167,1.311791e+21,5.221766e+11,23.236,1.778,2.770482e+11,1688.482,22.687,3.590,347.730865,26.670124,314,4094,0.000000,0.000000,0.000000,8.110000e-08,3.340000e-08,4.130000e-08,2.340000e-08,1100.0,32.9,0.0,4000.0,20.1,2.74,0.825,0.218
112877,7399,3.789936e+20,26.391,125.404,126.245,38.062,1.482250e+21,5.847634e+11,24.549,0.393,1.681806e+11,1722.400,23.677,5.093,345.256348,25.395241,299,4065,0.000000,0.002722,0.000607,8.200000e-08,3.380000e-08,3.960000e-08,2.810000e-08,1310.0,32.8,0.0,4360.0,23.2,2.57,0.683,0.152


In [33]:
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
metadata = {}


out_data = pd.read_csv("/mnt/hackathon2021/Weltraumwetterlage/train_data/output.csv")
out_data["Long"] = np.maximum(out_data["Gp_Long"].to_numpy(), out_data["Gs_Long"].to_numpy())
out_data["Short"] = np.maximum(out_data["Gp_Short"].to_numpy(), out_data["Gs_Short"].to_numpy())
out_data = out_data[["timestamp", "Long", "Short"]]
xray_df = out_data
xray_df = xray_df[(xray_df["Short"] > 0.0) & (xray_df["Long"] > 0.0)]

xray_columns = xray_df.columns
metadata["xray_mean"] = xray_df[xray_columns[1:]].mean().to_numpy()
metadata["xray_std"] = xray_df[xray_columns[1:]].std().to_numpy()

# balance input data 
much_boom_bang = xray_df[xray_df["Long"] >= 1.0e-7]
less_bomm_bang = xray_df[(xray_df["Long"] < 1.0e-7)].head(len(much_boom_bang))
ts_much_boom_bang = much_boom_bang["timestamp"]
ts_less_bomm_bang = less_bomm_bang["timestamp"]
x_ray_timestamps = ts_much_boom_bang.append(ts_less_bomm_bang)
xray_df = much_boom_bang.append(less_bomm_bang)

xray_df_notime = xray_df[xray_columns[1:]]
xray_df_notime -= metadata["xray_mean"]
xray_df_notime += metadata["xray_std"]
xray_df_notime["timestamp"] = x_ray_timestamps
xray_df = xray_df_notime
xray_df = xray_df.sort_values("timestamp")
x_ray_timestamps = x_ray_timestamps.sort_values()



sharp_df = pd.read_csv("/mnt/hackathon2021/Weltraumwetterlage/train_data/input.csv")
sharp_columns = sharp_df.columns
metadata["sharp_mean"] = sharp_df[sharp_columns[1:]].mean().to_numpy()
metadata["sharp_std"] = sharp_df[sharp_columns[1:]].std().to_numpy()

# balance input data
sharp_df["keep"] = sharp_df["timestamp"].apply(lambda x: x in x_ray_timestamps.values)
sharp_df = sharp_df[sharp_df["keep"] == False]
sharp_df_notime = sharp_df.drop(columns=["keep", "timestamp"])

sharp_df_notime -= metadata["sharp_mean"]
sharp_df_notime /= metadata["sharp_std"]
sharp_df_notime["timestamp"] = sharp_df["timestamp"]
sharp_df = sharp_df_notime



data = sharp_df.merge(xray_df, on='timestamp')

train_data = []


def create_train_data_tuple():
    random_time = datetime.fromtimestamp(int(data.sample(1)["timestamp"]))
    start, end = (random_time -
                  timedelta(hours=1)).timestamp(), random_time.timestamp()
    tmp_data = data[(data["timestamp"] > start) & (data["timestamp"] <= end)]
    harps = tmp_data["harp"].unique()

    input_ = np.zeros(INPUT_DIM)
    output = np.zeros(OUTPUT_DIM)

    for i, harp in enumerate(harps):
        if i == MAX_N_HARPS:
            break
        harp_data = tmp_data[tmp_data["harp"] == harp]
        input_data = harp_data[sharp_columns[1:]].to_numpy()
        n_timesteps = input_data.shape[0]
        if i == 0:
            output_data = harp_data[["Short", "Long"]].to_numpy()
            output[:n_timesteps, :] = output_data

        input_[:n_timesteps, i, :] = input_data

    return input_, output


def get_data(size=1000):
    X, y = [], []
    for _ in tqdm(range(size)):
        X_, y_ = create_train_data_tuple()
        X.append(X_)
        y.append(y_)
    return X, y

X, y = get_data(1000)
X_val, y_val = get_data(100)

print(np.any(np.isnan(X)))
print(np.any(np.isnan(y)))
print(np.any(np.isnan(X_val)))
print(np.any(np.isnan(y_val)))

TypeError: unhashable type: 'Series'

In [6]:
import tensorflow as tf
from tqdm import tqdm 


loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam()

BATCH_SIZE = 10
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    N = len(X) // BATCH_SIZE
    for batch in tqdm(range(N)):
        X_batch, y_batch = X[epoch*BATCH_SIZE:(epoch+1)*BATCH_SIZE], y[epoch*BATCH_SIZE:(epoch+1)*BATCH_SIZE]
        X_batch, y_batch = np.array(X_batch), np.array(y_batch)
        
        assert not np.any(np.isnan(X_batch)), "X_batch should not include nan"
        assert not np.any(np.isnan(y_batch)), "y_batch should not include nan"
        
        with tf.GradientTape() as tape:
            y_pred, _ = t((X_batch, y_batch), training=False)
        
            assert not np.any(np.isnan(y_pred)), "y_pred should not include nan"
        
            loss_value = loss(y_batch, y_pred)
        
        gradients = tape.gradient(loss_value, t.trainable_variables)
        optimizer.apply_gradients(zip(gradients, t.trainable_variables))
        
        total_loss += loss_value.numpy()

    print(f"EPOCH {epoch}: mean_loss = {total_loss / N}")
    
    y_pred_val, _ = t((np.array(X_val), np.array(y_val)))
    val_loss = loss(y_val, y_pred_val).numpy()
    
    print(f"EPOCH {epoch}: val_loss = {val_loss}")


100%|██████████| 100/100 [00:11<00:00,  8.81it/s]


EPOCH 0: mean_loss = 0.02934499163136934
EPOCH 0: val_loss = 0.008480074815452099


100%|██████████| 100/100 [00:11<00:00,  8.83it/s]


EPOCH 1: mean_loss = 0.0009690257180045591
EPOCH 1: val_loss = 0.004246722441166639


100%|██████████| 100/100 [00:11<00:00,  8.80it/s]


...

EPOCH 7: mean_loss = 3.4802185582520905e-05
EPOCH 7: val_loss = 0.00041802646592259407


100%|██████████| 100/100 [00:11<00:00,  8.84it/s]


EPOCH 8: mean_loss = 5.3405379512696525e-05
EPOCH 8: val_loss = 0.0002853221958503127


100%|██████████| 100/100 [00:11<00:00,  8.76it/s]


EPOCH 9: mean_loss = 4.736952485473012e-05
EPOCH 9: val_loss = 0.00035555148497223854


In [8]:
import pickle

MODELNAME = "8h-2"

metadata["n_timesteps"]  = N_TIMESTEPS
metadata["max_n_harps"]  = MAX_N_HARPS
metadata["n_features"] = N_FEATURES
metadata["n_out"]  = N_OUT
metadata["num_layers"] = NUM_LAYERS
metadata["d_model"] = D_MODEL
metadata["num_heads"] = NUM_HEADS
metadata["dff"] = DFF
metadata["input_dim"] = INPUT_DIM
metadata["output_dim"] = OUTPUT_DIM
metadata["rate"] = RATE



t.save_weights("/mnt/hackathon2021/modelcache/models/"+MODELNAME)
with open("/mnt/hackathon2021/modelcache/meta/"+MODELNAME+".pkl", "wb") as metafile:
    metafile.write(pickle.dumps(metadata))


In [4]:
class Avocato():
    def __init__(self, modelname):
        self.metadata = pickle.load(
            open("/mnt/hackathon2021/modelcache/meta/" + modelname + ".pkl",
                 "rb"))
        self.model = Transformer(
            num_layers=self.metadata["num_layers"],
            d_model=self.metadata["d_model"],
            num_heads=self.metadata["num_heads"],
            dff=self.metadata["dff"],
            input_dimensions=self.metadata["input_dim"],
            target_dimensions=self.metadata["output_dim"],
            rate=self.metadata["rate"],
        )
        self.model.compile()
        self.model.load_weights("/mnt/hackathon2021/modelcache/models/" +
                                modelname)

    def __call__(self, net_in):
        """
        Args:
        -----
            - net_in: pd dataframe consisting of n_timesteps of data for HARPS (timestamp, harp, ...parameters)
        """

        in_data = np.zeros(
            (1, self.metadata["n_timesteps"], self.metadata["max_n_harps"],
             self.metadata["n_features"]))
        unique_timesteps = net_in["timestamp"].unique()
        for t_id, timestep in enumerate(unique_timesteps):
            if t_id >= self.metadata["n_timesteps"]:
                continue
            unique_harps = net_in[net_in["timestamp"] ==
                                  timestep]["harp"].unique()
            for h_id, harp in enumerate(unique_harps):
                if h_id >= self.metadata["max_n_harps"]:
                    continue
                in_data[0, t_id, h_id] = net_in[
                    (net_in["timestamp"] == timestep) &
                    (net_in["harp"] == harp
                     )].loc[:,
                            net_in.columns != "timestamp"].to_numpy().reshape(
                                self.metadata["n_features"])
                in_data[0, t_id, h_id] -= self.metadata["sharp_mean"]
                in_data[0, t_id, h_id] /= self.metadata["sharp_std"]

        output = np.zeros((
            1,
            self.metadata["n_timesteps"],
            self.metadata["n_out"],
        ))

        out_list = []
        for i in tf.range(self.metadata["n_timesteps"]):
            output, _ = self.model([in_data, output], training=False)
            out_list.append(output * self.metadata["xray_std"] +
                            self.metadata["xray_mean"])

        return out_list


In [6]:
import pickle 

MODELNAME = "8h-2"
pred = Avocato(MODELNAME)
random_time = datetime.fromtimestamp(int(data.sample(1)["timestamp"]))
start, end = (random_time -
                timedelta(hours=1)).timestamp(), random_time.timestamp()
net_in = data[(data["timestamp"] > start) & (data["timestamp"] <= end)]
net_in = net_in[sharp_columns]

In [8]:
import tensorflow as tf

timestamps = data["timestamp"].unique()
predictions = []
for ts in tqdm(timestamps[10:20]):
    random_time = datetime.fromtimestamp(int(ts))
    start, end = (random_time -
                timedelta(hours=1)).timestamp(), random_time.timestamp()
    net_in = data[(data["timestamp"] > start) & (data["timestamp"] <= end)]
    net_in = net_in[sharp_columns]
    predictions.append(tf.math.reduce_max(tf.math.reduce_max(pred(net_in), axis=-2), axis=0).numpy()[0][-1])

100%|██████████| 10/10 [00:02<00:00,  3.66it/s]


In [40]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(x=list(range(len(predictions))), y=predictions))
fig.show()

In [9]:
tf.math.reduce_max(pred(net_in), axis=-2).numpy()[0][-1]

array([2.30374919e-08, 1.17871096e-07], dtype=float32)

In [16]:
tf.math.reduce_max(tf.math.reduce_max(pred(net_in), axis=-2), axis=0).numpy()[0][-1]

1.8530316e-07